In [0]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_selection import RFE

import warnings
import numpy as np
import pandas as pd
import time
from xgboost import XGBRegressor, plot_importance


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_df = pd.read_csv("/content/drive/My Drive/CE4032/dataset/modified_train_v3.csv")
test_df = pd.read_csv("/content/drive/My Drive/CE4032/dataset/modified_test_v3.csv")
ids = test_df['TRIP_ID']
train_df.head()


,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DATE,END_TIME,DAY_OF_WEEK,HOUR,ORIGIN_LNG,ORIGIN_LAT,ORIGIN_HEADER,ORIGIN_DISTANCE_TO_CC,CUT_OFF_LENGTH,CUT_OFF_LAT,CUT_OFF_LNG,CUT_OFF_DIST_FROM_CC,HEADER_CUT_OFF_TO_CC,CUT_OFF_DIST_FROM_ORIGIN,HEADER_ORIGIN_TO_CUT_OFF,CUM_DIST,MEDIAN_VELOCITY,FINAL_VELOCITY,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ACTUAL_DAYTYPE_A,ACTUAL_DAYTYPE_B,ACTUAL_DAYTYPE_C,DURATION
0,1372636858620000589,NaN,NaN,20000589,1372636858,2013-07-01 01:00:58+01:00,1372637188,0,8,-8.618643,41.141412,0.149319,1830.976883,19,41.154507,-8.630829,1341.015903,2.185836,1780.005072,-0.611154,2623.280294,11.041239,12.725186,0,0,1,1,0,0,330
1,1372636858620000589,NaN,NaN,20000589,1372636858,2013-07-01 01:00:58+01:00,1372637188,0,8,-8.618643,41.141412,0.149319,1830.976883,4,41.144373,-8.623953,1645.635018,2.185903,553.896759,-0.933391,422.296381,13.232000,14.073105,0,0,1,1,0,0,330
2,1372637303620000596,NaN,7.0,20000596,1372637303,2013-07-01 01:08:23+01:00,1372637573,0,8,-8.639847,41.159826,1.686860,2063.429934,7,41.162580,-8.656434,3482.655006,2.186018,1423.528654,-1.353639,1143.447103,13.561319,18.148117,0,1,0,1,0,0,270
3,1372636951620000320,NaN,NaN,20000320,1372636951,2013-07-01 01:02:31+01:00,1372637911,0,8,-8.612964,41.140359,-0.105214,1938.539952,11,41.145174,-8.630082,1858.247962,2.185963,1531.865239,-1.213231,1368.236297,9.194587,13.289274,0,0,1,1,0,0,960
4,1372636951620000320,NaN,NaN,20000320,1372636951,2013-07-01 01:02:31+01:00,1372637911,0,8,-8.612964,41.140359,-0.105214,1938.539952,17,41.152464,-8.628579,1248.274734,2.185839,1878.613544,-0.770793,2534.477061,12.578108,11.892797,0,0,1,1,0,0,960


In [0]:
train_df['DURATION'] = train_df['DURATION'] / 15
train_cl = train_df.drop(columns=['TRIP_ID','TIMESTAMP','DATE', 'END_TIME',  'DURATION', 'ORIGIN_CALL',	'ORIGIN_STAND'])
test_cl= test_df.drop(columns=[ 'TRIP_ID','TIMESTAMP','DATE', 'END_TIME', 'ORIGIN_CALL',	'ORIGIN_STAND'])
train_cl.head()

,TAXI_ID,DAY_OF_WEEK,HOUR,ORIGIN_LNG,ORIGIN_LAT,ORIGIN_HEADER,ORIGIN_DISTANCE_TO_CC,CUT_OFF_LENGTH,CUT_OFF_LAT,CUT_OFF_LNG,CUT_OFF_DIST_FROM_CC,HEADER_CUT_OFF_TO_CC,CUT_OFF_DIST_FROM_ORIGIN,HEADER_ORIGIN_TO_CUT_OFF,CUM_DIST,MEDIAN_VELOCITY,FINAL_VELOCITY,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ACTUAL_DAYTYPE_A,ACTUAL_DAYTYPE_B,ACTUAL_DAYTYPE_C
0,20000589,0,8,-8.618643,41.141412,0.149319,1830.976883,19,41.154507,-8.630829,1341.015903,2.185836,1780.005072,-0.611154,2623.280294,11.041239,12.725186,0,0,1,1,0,0
1,20000589,0,8,-8.618643,41.141412,0.149319,1830.976883,4,41.144373,-8.623953,1645.635018,2.185903,553.896759,-0.933391,422.296381,13.232000,14.073105,0,0,1,1,0,0
2,20000596,0,8,-8.639847,41.159826,1.686860,2063.429934,7,41.162580,-8.656434,3482.655006,2.186018,1423.528654,-1.353639,1143.447103,13.561319,18.148117,0,1,0,1,0,0
3,20000320,0,8,-8.612964,41.140359,-0.105214,1938.539952,11,41.145174,-8.630082,1858.247962,2.185963,1531.865239,-1.213231,1368.236297,9.194587,13.289274,0,0,1,1,0,0
4,20000320,0,8,-8.612964,41.140359,-0.105214,1938.539952,17,41.152464,-8.628579,1248.274734,2.185839,1878.613544,-0.770793,2534.477061,12.578108,11.892797,0,0,1,1,0,0


In [0]:
# values = {'ORIGIN_CALL': -1, 'ORIGIN_STAND': -1}
# train_cl = train_cl.fillna(value=values)
# test_cl = test_cl.fillna(value=values)
test_cl.head()

,TAXI_ID,DAY_OF_WEEK,HOUR,ORIGIN_LNG,ORIGIN_LAT,ORIGIN_HEADER,ORIGIN_DISTANCE_TO_CC,CUT_OFF_LENGTH,CUT_OFF_LAT,CUT_OFF_LNG,CUT_OFF_DIST_FROM_CC,HEADER_CUT_OFF_TO_CC,CUT_OFF_DIST_FROM_ORIGIN,HEADER_ORIGIN_TO_CUT_OFF,CUM_DIST,MEDIAN_VELOCITY,FINAL_VELOCITY,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ACTUAL_DAYTYPE_A,ACTUAL_DAYTYPE_B,ACTUAL_DAYTYPE_C
0,20000542,3,18,-8.585676,41.148522,-1.182260,2691.258243,11,41.146623,-8.584884,2837.910054,2.019103,221.575614,2.837288,513.623887,2.411640,10.243918,0,1,0,0,0,1
1,20000108,3,18,-8.610876,41.145570,-0.273847,1399.904997,40,41.163597,-8.601894,1309.291868,1.043254,2143.324472,0.358853,2282.133126,3.340840,5.146115,0,1,0,0,0,1
2,20000370,3,18,-8.585739,41.148558,-1.182897,2684.853296,40,41.167719,-8.574903,3572.711105,1.252324,2318.288447,0.402469,3911.607406,5.660043,6.645085,0,1,0,0,0,1
3,20000492,3,18,-8.613963,41.141169,-0.065131,1841.553918,8,41.140980,-8.614638,1859.765020,3.107550,60.372488,-1.926755,176.758074,1.064723,2.607521,0,1,0,0,0,1
4,20000621,3,18,-8.619903,41.148036,0.338632,1137.845589,2,41.148036,-8.619894,1137.595194,-2.803534,0.754424,1.570796,0.754424,0.050295,0.050295,0,1,0,0,0,1


In [0]:
train = train_cl


In [0]:
test = test_cl


In [0]:
feature_names = list(train.columns)

In [0]:
y= np.log(train_df['DURATION']) + 1

In [54]:
Xtr, Xv, ytr, yv = train_test_split(train[feature_names].values, y, test_size=0.2, random_state=1987)
dtrain = xgb.DMatrix(Xtr, label=ytr)
dvalid = xgb.DMatrix(Xv, label=yv)
dtest = xgb.DMatrix(test[feature_names].values)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

#best
xgb_pars = {'min_child_weight': 10, 'eta': 0.15, 'colsample_bytree': 0.5, 'max_depth': 15, 'subsample': 0.9, 'lambda': 2.0, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}




/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [55]:
model = xgb.train(xgb_pars, dtrain, 10, watchlist, early_stopping_rounds=10,
                  maximize=False, verbose_eval=1)

[0]	train-rmse:3.64547	valid-rmse:3.6452
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[1]	train-rmse:3.10646	valid-rmse:3.10632
[2]	train-rmse:2.64993	valid-rmse:2.64988
[3]	train-rmse:2.26276	valid-rmse:2.26288
[4]	train-rmse:1.9354	valid-rmse:1.93548
[5]	train-rmse:1.6596	valid-rmse:1.65991
[6]	train-rmse:1.42661	valid-rmse:1.42725
[7]	train-rmse:1.23018	valid-rmse:1.23125
[8]	train-rmse:1.0664	valid-rmse:1.06811
[9]	train-rmse:0.928594	valid-rmse:0.931454


In [56]:
logytest = model.predict(dtest)
ytest = (np.exp(logytest) -1 )
print('Test shape OK.') if test.shape[0] == ytest.shape[0] else print('Oops')
result= pd.DataFrame({"TRIP_ID": test_df["TRIP_ID"], "TRAVEL_TIME": ytest*15})
result.to_csv("/content/drive/My Drive/CE4032/dataset/20tree_param_10epoch_xgboost_regression.csv", index=False)

Test shape OK.


The following section was used to find the best parameters for training.

In [0]:
FOREVER_COMPUTING_FLAG = True
xgb_pars = []
for MCW in [10, 20, 50, 75, 100]:
    for ETA in [0.05, 0.1, 0.15]:
        for CS in [0.3, 0.4, 0.5]:
            for MD in [6, 8, 10, 12, 15]:
                for SS in [0.5, 0.6, 0.7, 0.8, 0.9]:
                    for LAMBDA in [0.5, 1., 1.5,  2., 3.]:
                        xgb_pars.append({'min_child_weight': MCW, 'eta': ETA, 
                                         'colsample_bytree': CS, 'max_depth': MD,
                                         'subsample': SS, 'lambda': LAMBDA, 
                                         'nthread': -1, 'booster' : 'gbtree', 'eval_metric': 'rmse',
                                         'silent': 1, 'objective': 'reg:linear'})
# count= 0
while FOREVER_COMPUTING_FLAG:
    xgb_par = np.random.choice(xgb_pars, 1)[0]
    print(xgb_par)
    model = xgb.train(xgb_par, dtrain, 10, watchlist, early_stopping_rounds=50,
                      maximize=False, verbose_eval=10)
    # nametouse = "/content/drive/My Drive/CE4032/dataset/" + str(count) + ".csv"
    print('Modeling RMSLE %.5f' % model.best_score)
    # logytest = model.predict(dtest)
    # ytest = (np.exp(logytest) -1 ) * 15
    # print('Test shape OK.') if test.shape[0] == ytest.shape[0] else print('Oops')
    # result= pd.DataFrame({"TRIP_ID": test_df["TRIP_ID"], "TRAVEL_TIME": ytest})
    # result.to_csv(str(nametouse), index=False)
    # count +=1


{'min_child_weight': 75, 'eta': 0.1, 'colsample_bytree': 0.3, 'max_depth': 10, 'subsample': 0.5, 'lambda': 2.0, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:3.88171	valid-rmse:3.88253
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[9]	train-rmse:1.5606	valid-rmse:1.56113
Modeling RMSLE 1.56113
{'min_child_weight': 50, 'eta': 0.1, 'colsample_bytree': 0.4, 'max_depth': 6, 'subsample': 0.6, 'lambda': 2.0, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:3.87926	valid-rmse:3.88059
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[9]	train-rmse:1.55643	valid-rmse:1.55686
Modeling RMSLE 1.55686
{'min_child_weight': 75, 'eta': 0.15, 'colsample_bytree': 0.5, 'max_depth': 

[9]	train-rmse:0.934293	valid-rmse:0.935502
{'min_child_weight': 50, 'eta': 0.15, 'colsample_bytree': 0.5, 'max_depth': 10, 'subsample': 0.8, 'lambda': 1.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}


{'min_child_weight': 10, 'eta': 0.15, 'colsample_bytree': 0.5, 'max_depth': 12, 'subsample': 0.9, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
train-rmse:0.933942 Modeling RMSLE 0.93526
